In [127]:
from stellargraph.mapper import CorruptedGraphSAGENodeGenerator
import stellargraph as sg
import networkx as nx
from stellargraph import StellarGraph, StellarDiGraph
from stellargraph.layer import GraphSAGEInfoMax
from stellargraph import datasets
import random
import numpy as np
from sklearn import preprocessing, feature_extraction, model_selection

from IPython.display import display, HTML

from tensorflow.keras.layers import Input, Dense
import tensorflow as tf
from tensorflow.keras import Model

In [5]:
def info_loss(y_true, y_pred):
    
    return tf.math.reduce_mean(y_pred)  

In [125]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, node_subjects = dataset.load()

In [128]:
train_subjects, test_subjects = model_selection.train_test_split(
    node_subjects, train_size=0.1, test_size=None, stratify=node_subjects
)

target_encoding = preprocessing.LabelBinarizer()

train_targets = target_encoding.fit_transform(train_subjects)
test_targets = target_encoding.transform(test_subjects)

In [131]:
batch_size = 50
num_samples = [10, 5]

generator = CorruptedGraphSAGENodeGenerator(G, batch_size, num_samples)
train_gen = generator.flow(train_subjects.index, targets=np.ones(len(train_subjects)))

graph_sage_infomax = GraphSAGEInfoMax([20, 20,], generator=generator)

x_in, x_out = graph_sage_infomax.unsupervised_node_model()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=info_loss, optimizer='adam')
model.fit(train_gen, epochs=50)

  ...
    to  
  ['...']
Train for 6 steps
Epoch 1/50
6/6 [==============================] - 1s 172ms/step - loss: 0.4952
Epoch 2/50
6/6 [==============================] - 0s 48ms/step - loss: 0.4765
Epoch 3/50
6/6 [==============================] - 0s 45ms/step - loss: 0.4538
Epoch 4/50
6/6 [==============================] - 0s 47ms/step - loss: 0.4308
Epoch 5/50
6/6 [==============================] - 0s 46ms/step - loss: 0.3874
Epoch 6/50
6/6 [==============================] - 0s 46ms/step - loss: 0.3506
Epoch 7/50
6/6 [==============================] - 0s 46ms/step - loss: 0.3091
Epoch 8/50
6/6 [==============================] - 0s 45ms/step - loss: 0.2712
Epoch 9/50
6/6 [==============================] - 0s 46ms/step - loss: 0.2640
Epoch 10/50
6/6 [==============================] - 0s 44ms/step - loss: 0.2303
Epoch 11/50
6/6 [==============================] - 0s 44ms/step - loss: 0.1953
Epoch 12/50
6/6 [==============================] - 0s 46ms/step - loss: 0.1737
Epoch 13/50
6/6 [

In [142]:
model.layers

In [135]:
x_in, x_out = graph_sage_infomax.unsupervised_node_model()

model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=info_loss, optimizer='adam')

model.evaluate(train_gen)

  ...
    to  
  ['...']
6/6 [==============================] - 0s 77ms/step - loss: 0.4634


0.46338745454947156

In [132]:
test_gen = generator.flow(test_subjects.index)
test_embeddings = model.predict(test_gen)
train_embeddings = 

In [133]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit()
test_targets

626530      Probabilistic_Methods
714975      Probabilistic_Methods
820661            Neural_Networks
6216       Reinforcement_Learning
126868                 Case_Based
                    ...          
1115677           Neural_Networks
1131639           Neural_Networks
672070            Neural_Networks
1107136        Genetic_Algorithms
1128853             Rule_Learning
Name: subject, Length: 2438, dtype: object